In [10]:
from universal_format import UniversalFormat
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

In [5]:
p = Path("/home/mcormier/git/divan/tests/large_files")
barcode = 181150
f = "AE_CYC_181150_Nw_43V_c0_40C_1C_210127_NMC640_25DOD_1C_127.0.0.1_230115_4_4.txt"
fpath = p / f
fpath = str(fpath)
print(fpath)

/home/mcormier/git/divan/tests/large_files/AE_CYC_181150_Nw_43V_c0_40C_1C_210127_NMC640_25DOD_1C_127.0.0.1_230115_4_4.txt


If you are on lab network, you should be able to access files from "Y:\databdayse\DATA\CYCLING\NEWARE\NEWAREB12B". Just specify that inside Path(). 

I use the "barcode" to name the output files later.

In [ ]:
data = UniversalFormat(fpath)

### Get (time, discharge capacity) and (cycle number for full C/20 cycles

In [7]:
rate = "C/20"
rate_cycs = data.select_by_rate(rate, cyctype='cyc')
print(' First cycle found: {0:d} \n Last cycle found: {1:d} \n Number of cycles at {2}: {3:d}'.format(rate_cycs[0], rate_cycs[-1], rate, len(rate_cycs)))

 First cycle found: 1 
 Last cycle found: 17867 
 Number of cycles at C/20: 36


In [9]:
c, dV = data.deltaV(cycnums=rate_cycs)
print(len(c), len(dV))
print(dV)

18 18
[4.5907731876538116e-08, 5.0149284337904954e-08, 4.865202047587166e-08, 4.6697544385248265e-08, 4.8861958292693013e-08, 5.022467424161858e-08, 5.26416787970071e-08, 5.145022002940613e-08, 5.425494339946686e-08, 5.50792552315042e-08, 5.373305388480252e-08, 5.5652007730530156e-08, 5.59623696923471e-08, 5.750997738391002e-08, 5.7677731829105176e-08, 5.71438664664847e-08, 5.740000355384963e-08, 5.962977801778012e-08]


#### Delta V sometimes fails due to division by zero. Use the good cycles returned by data.deltaV() to get corresponding discharge capacities.

In [ ]:
t, discap  = data.get_discap(x_var='time', cycnums=c)
print(len(t), len(discap))

#### Put (cycle number, time, discharge capacity, delta V) in dataframe and save to csv file.

In [ ]:
df = pd.DataFrame(data={"Time": t, "Cycle": c, "Discharge Capacity": discap, "Delta V": dV})
dstpath = Path("/home/mcormier/")
df.to_csv(path_or_buf=dstpath / "Cover20_time-cyc-discap-dV_{}.csv".format(barcode))

### Can do the operations all at once, looping over rates.

In [ ]:
rates = ["C/20", "1C"]
rates_for_file = ["Cover20", "1C"]
for i in range(len(rates)):
    print("Gettin {} cycles.".format(rates[i]))
    rate_cycs = data.select_by_rate(rates[i], cyctype='cyc')
    print(' First cycle found: {0:d} \n Last cycle found: {1:d} \n Number of cycles at {2}: {3:d}'.format(rate_cycs[0], rate_cycs[-1], rates[i], len(rate_cycs)))
    c, dV = data.deltaV(cycnums=rate_cycs)
    print(" Found {0:d} good delta V calculations.".format(len(c)))
    t, discap  = data.get_discap(x_var='time', cycnums=c)
    df = pd.DataFrame(data={"Time": t, "Cycle": c, "Discharge Capacity": discap, "Delta V": dV})
    dstpath = Path("/home/mcormier/")
    df.to_csv(path_or_buf=dstpath / "{0}_time-cyc-discap-dV_{1}.csv".format(rates_for_file[i], barcode))